# Teste 1 Modelo para Todo DataSet

In [1]:
%run ../src/feature_engineering.py
%run ../src/model_evaluation.py
%run ../src/plot.py
%run ../src/pipeline.py

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras import models
from keras import layers
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline
#Alterando configurações padrão dos plots
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20, 160)

Using TensorFlow backend.


In [2]:
course_df = pd.read_csv('../data/preprocessed/edx_2014_train.csv',low_memory=False)


In [4]:
plt.rcParams['figure.figsize'] = (20, 160)

class ModelDecorator:
    def __init__(self, model):
        self._model = model
        
    def fit(self, X, y):
        return self._model.fit(X, y, epochs=50, verbose=1)
    
    def predict(self, X):
        flatten = lambda l: [item for sublist in l for item in sublist]
        return flatten(self._model.predict(X))

class _LinearRegression:
    def provide(self):
        return LinearRegression()
    def sample_size(self):
        return None
    
class _DecisionTree:
    def provide(self):
        return DecisionTreeRegressor(criterion='mae', max_depth=3)
    def sample_size(self):
        return None
    
class _DeepLearning:
    def provide(self):
        model = models.Sequential()
        model.add(layers.Dense(32, activation='relu', input_dim=8))
        model.add(layers.Dense(32, activation='relu'))     
        model.add(layers.Dense(1))
        model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
        return ModelDecorator(model)
    def sample_size(self):
        return 50000
    
class _SupportVectorMachine:
    def provide(self):
        return Pipeline([('scaler', StandardScaler()),
                  ('poly', SVR(kernel='poly', C=1e3, cache_size=7000, verbose=True))])
    def sample_size(self):
        return 1
        
class _RandonForest:
    def provide(self):
        return Pipeline([('scaler', StandardScaler()),
                  ('rfr', RandomForestRegressor(criterion='mae',  max_depth=3))])
    def sample_size(self):
        return 10000

_models = [    
  #  _SupportVectorMachine(),
    _DecisionTree(),
    _RandonForest(),
    _LinearRegression(),
    _DeepLearning(),
    
]
    
#course_df = course_df.dropna(subset=['grade'])   
#course_df['grade'] = course_df['grade'].map(lambda g: g if g > 0 else 0)
    
def feat_extract_selected(df):
    data = feature_extract(df)
    return data[['grade',
                 'viewed',
                 'explored',
                 'certified',
                 'nevents',
                 'ndays_act',
                 'nplay_video',
                 'nchapters',
                 'nforum_posts']]

results, plt = train_evaluate_by_course(_models, course_df, feature_extractor= feat_extract_selected, 
                              label='grade', split_criteria='course_id', evaluation=regression_evaluation, 
                              plots= [scatter_real_vs_pred_subplot, scatter_residual_error_subplot])

plt.show()

2019-05-31 07:18:51 Iniciando modelo _DecisionTree


KeyError: "['certified'] not in index"

In [ ]:
reform = {(level1_key, level2_key, level3_key): values
              for level1_key, level2_dict in results.items()
              for level2_key, level3_dict in level2_dict.items()
              for level3_key, values      in level3_dict.items()}

result2 = []
for modelname, courses in results.items():
    for course_id, metrics in courses.items():
        result2.append({'model': modelname, 'course_id': course_id, 'mae':metrics['mean_absolute_error']})
    
pd.options.display.float_format = '{:,.4f}'.format
pd.DataFrame(result2)

In [5]:
reform = {(level1_key, level2_key, level3_key): values
              for level1_key, level2_dict in results.items()
              for level2_key, level3_dict in level2_dict.items()
              for level3_key, values      in level3_dict.items()}

result2 = []
for modelname, courses in results.items():
    for course_id, metrics in courses.items():
        result2.append({'model': modelname, 'course_id': course_id, 'r2_score':metrics['r2_score']})
    
pd.options.display.float_format = '{:,.4f}'.format
pd.DataFrame(result2)

NameError: name 'results' is not defined